In [206]:
from datetime import timedelta

import pandas as pd
from openpyxl import load_workbook
from openpyxl.styles import (
    Border,
    Font,
    PatternFill,
    Side,
    Alignment,
)

In [207]:
# General restrictions:
start_date = "2025-01-01"
num_days = 365

employee_restrictions = {
    "hours_per_shift": 7,
    "max_hours_week_employee": 37.5,
    "max_hours_year_employee": 1852.5,  # TODO: Pending to check
    "min_weekend_rest_month_employee": 1,
    "max_timeoff_employee": 2,
    "shifts": ["M", "T"],
    "max_persons_per_shift": {
        "M": 1,
        "T": 1,
    },
}

In [208]:
# Employees information
employees_full = [
    {
        "capacity": 1,
    }
]

employees_partial = [
    {
        "capacity": 0.77,
    }
]

employees_temp = employees_full * 2 + employees_partial * 1

employees = []
for index, one_employee in enumerate(employees_temp):
    employees.append(
        {
            "name": f"E{index + 1}",
            "capacity": one_employee["capacity"],
            "max_hours_year":
                employee_restrictions["max_hours_year_employee"]
                * one_employee["capacity"]
            ,
            "max_hours_week":
                employee_restrictions["max_hours_week_employee"]
                * one_employee["capacity"]
            ,
        }
    )

employees

[{'name': 'E1',
  'capacity': 1,
  'max_hours_year': 1852.5,
  'max_hours_week': 37.5},
 {'name': 'E2',
  'capacity': 1,
  'max_hours_year': 1852.5,
  'max_hours_week': 37.5},
 {'name': 'E3',
  'capacity': 0.77,
  'max_hours_year': 1426.425,
  'max_hours_week': 28.875}]

In [209]:
# Init employees with dates:
dates = pd.date_range(start=start_date, periods=num_days, freq="D")
employees_info = pd.DataFrame(
    index=dates, columns=[emp["name"] for emp in employees], data=""
)
employees_info

,E1,E2,E3
2025-01-01,,,
2025-01-02,,,
2025-01-03,,,
2025-01-04,,,
2025-01-05,,,
...,...,...,...
2025-12-27,,,
2025-12-28,,,
2025-12-29,,,
2025-12-30,,,


In [210]:
# Init all employees by shift
all_employees_by_shift = pd.DataFrame(
    index=dates, columns=[one_shift for one_shift in employee_restrictions["shifts"]]
)
all_employees_by_shift[:] = 0
all_employees_by_shift

,M,T
2025-01-01,0,0
2025-01-02,0,0
2025-01-03,0,0
2025-01-04,0,0
2025-01-05,0,0
...,...,...
2025-12-27,0,0
2025-12-28,0,0
2025-12-29,0,0
2025-12-30,0,0


In [211]:
# Fill information:

for date in all_employees_by_shift.index:
    for shift in all_employees_by_shift.columns:
        if (
                all_employees_by_shift.loc[date, shift]
                >= employee_restrictions["max_persons_per_shift"][shift]
        ):  # No more employees needed
            continue

        available_employees = []

        for employee in employees_info.columns:
            if not employees_info.loc[date, employee]:
                six_days_ago = date - timedelta(days=6)
                five_days_ago = date - timedelta(days=5)
                thirty_days_ago = date - timedelta(days=30)

                last_6_days_employee = employees_info.loc[six_days_ago:date, employee]
                last_5_days_employee = employees_info.loc[five_days_ago:date, employee]
                last_30_days_employee = employees_info.loc[thirty_days_ago:date, employee]

                total_worked_days_in_6_days = last_6_days_employee.value_counts().reindex(
                    ["M", "T"], fill_value=0
                ).sum()
                total_rest_days_in_5_days = last_5_days_employee.value_counts().reindex(
                    ["-"], fill_value=0
                ).sum()

                weekend_days = last_30_days_employee[last_30_days_employee.index.weekday.isin([6])]
                total_worked_weekends_in_30_days = weekend_days.value_counts().reindex(["M", "T"], fill_value=0).sum()

                employee_capacity = next(
                    emp["capacity"] for emp in employees if emp["name"] == employee
                )
                if (total_rest_days_in_5_days >= employee_restrictions["max_timeoff_employee"]) or (
                        (total_worked_days_in_6_days + 1) * employee_restrictions[
                    "hours_per_shift"
                ] < employee_restrictions[
                            "max_hours_week_employee"
                        ] * employee_capacity):
                    available_employees.append(
                        {"employee": employee, "total_worked_weekends_in_30_days": total_worked_weekends_in_30_days})
                else:
                    employees_info.loc[date, employee] = "-"
        # Sort available_employees per total_worked_weekends_in_30_days - Descending order
        available_employees = sorted(available_employees, key=lambda x: x["total_worked_weekends_in_30_days"],
                                     reverse=False)
        for one_employee in available_employees:
            all_employees_by_shift.loc[date, shift] += 1
            employees_info.loc[date, one_employee["employee"]] = shift
            if (
                    all_employees_by_shift.loc[date, shift]
                    >= employee_restrictions["max_persons_per_shift"][shift]
            ):  # No more employees needed
                break
    for one_employee in employees_info.columns:
        if employees_info.loc[date, one_employee] == "":
            employees_info.loc[date, one_employee] = "-"


In [212]:
all_employees_by_shift.index = pd.to_datetime(all_employees_by_shift.index)
all_employees_by_shift.index = all_employees_by_shift.index.strftime("%Y-%m-%d")
all_employees_by_shift

,M,T
2025-01-01,1,1
2025-01-02,1,1
2025-01-03,1,1
2025-01-04,1,1
2025-01-05,1,1
...,...,...
2025-12-27,1,1
2025-12-28,1,1
2025-12-29,1,1
2025-12-30,1,1


In [213]:
employees_info.index = pd.to_datetime(employees_info.index)
employees_info.index = employees_info.index.strftime("%Y-%m-%d")
employees_info

,E1,E2,E3
2025-01-01,M,T,-
2025-01-02,M,T,-
2025-01-03,M,T,-
2025-01-04,M,T,-
2025-01-05,M,T,-
...,...,...,...
2025-12-27,-,M,T
2025-12-28,M,-,T
2025-12-29,T,M,-
2025-12-30,T,M,-


In [214]:
employees_info.index = pd.to_datetime(employees_info.index)
employees_info.index = employees_info.index.strftime("%Y-%m-%d")
employees_info[["E1", "E2", "E3"]]

,E1,E2,E3
2025-01-01,M,T,-
2025-01-02,M,T,-
2025-01-03,M,T,-
2025-01-04,M,T,-
2025-01-05,M,T,-
...,...,...,...
2025-12-27,-,M,T
2025-12-28,M,-,T
2025-12-29,T,M,-
2025-12-30,T,M,-


In [215]:
output_filename = "samples/m_a_2025.xlsx"
employees_info.index = pd.to_datetime(employees_info.index)
employees_info.index = employees_info.index.strftime("%Y-%m-%d")
employees_info.to_excel(output_filename, sheet_name="Shift Schedule")

In [216]:
employees_info.index = pd.to_datetime(employees_info.index)

day_of_month = employees_info.index.day
days_of_week_map = {0: "L", 1: "M", 2: "X", 3: "J", 4: "V", 5: "S", 6: "D"}
months_map = {
    1: "Enero",
    2: "Febrero",
    3: "Marzo",
    4: "Abril",
    5: "Mayo",
    6: "Junio",
    7: "Julio",
    8: "Agosto",
    9: "Septiembre",
    10: "Octubre",
    11: "Noviembre",
    12: "Diciembre",
}
month = employees_info.index.month.map(months_map)
day_of_week = employees_info.index.dayofweek.map(days_of_week_map)

multi_index_index = pd.MultiIndex.from_arrays(
    [month, day_of_week, day_of_month], names=["", "", ""]
)

employees_info.index = multi_index_index

transposed_employees_info = employees_info.T
transposed_employees_info

Enero                             ... Diciembre                           
       X  J  V  S  D  L  M  X  J  V  ...         L  M  X  J  V  S  D  L  M  X
      1  2  3  4  5  6  7  8  9  10  ...        22 23 24 25 26 27 28 29 30 31
E1     M  M  M  M  M  -  -  T  T  M  ...         T  T  T  M  -  -  M  T  T  T
E2     T  T  T  T  T  -  -  -  -  T  ...         M  M  M  -  M  M  -  M  M  M
E3     -  -  -  -  -  M  M  M  M  -  ...         -  -  -  T  T  T  T  -  -  -

[3 rows x 365 columns]

In [217]:
transposed_employees_info["THT"] = transposed_employees_info.apply(lambda row: (
                                                                                       row.value_counts().get("M",
                                                                                                              0) + row.value_counts().get(
                                                                                   "T", 0)) * employee_restrictions[
                                                                                   "hours_per_shift"], axis=1)

transposed_employees_info["MH"] = transposed_employees_info.index.map(
    lambda emp: next(employee["max_hours_year"] for employee in employees if employee["name"] == emp)
)
transposed_employees_info["Diff"] = transposed_employees_info["MH"] - transposed_employees_info["THT"]
transposed_employees_info

Enero                             ... Diciembre                     THT  \
       X  J  V  S  D  L  M  X  J  V  ...         J  V  S  D  L  M  X         
       1  2  3  4  5  6  7  8  9 10  ...        25 26 27 28 29 30 31         
E1     M  M  M  M  M  -  -  T  T  M  ...         M  -  -  M  T  T  T  1806   
E2     T  T  T  T  T  -  -  -  -  T  ...         -  M  M  -  M  M  M  1820   
E3     -  -  -  -  -  M  M  M  M  -  ...         T  T  T  T  -  -  -  1456   

          MH    Diff  
                      
                      
E1  1852.500  46.500  
E2  1852.500  32.500  
E3  1426.425 -29.575  

[3 rows x 368 columns]

In [218]:
output_filename = "samples/m_a_2025_transpose.xlsx"
transposed_employees_info.to_excel(output_filename, sheet_name="Shift Schedule")

workbook = load_workbook(output_filename)
worksheet = workbook["Shift Schedule"]

worksheet.delete_rows(4)

min_width = 3
for col in worksheet.iter_cols():
    for cell in col:
        if not any(
                cell.coordinate in merged_cell
                for merged_cell in worksheet.merged_cells.ranges
        ):
            column = cell.column_letter
            worksheet.column_dimensions[column].width = min_width
            break

for i in range(0,3):
    column_letter = worksheet.cell(row=1, column=worksheet.max_column - i).column_letter
    worksheet.column_dimensions[column_letter].width = 7
    for cell in worksheet[column_letter]:
        cell.alignment = Alignment(horizontal="center")

fill = PatternFill(start_color="0099FF", end_color="0099FF", fill_type="solid")
font = Font(color="FFFFFF", bold=True)

for cell in worksheet[1]:
    cell.fill = fill
    cell.font = font

weekend_fill = PatternFill(start_color="FFFF00", end_color="FFFF00", fill_type="solid")

thin_border = Border(
    left=Side(style="thin"),
    right=Side(style="thin"),
    top=Side(style="thin"),
    bottom=Side(style="thin"),
)

for col in worksheet.iter_cols(
        min_row=2, max_row=worksheet.max_row, min_col=2, max_col=worksheet.max_column
):
    day_of_week_cell = col[0]
    if day_of_week_cell.value in ["S", "D"]:
        for cell in col:
            cell.fill = weekend_fill

for row in worksheet.iter_rows(
        min_row=1, max_row=worksheet.max_row, min_col=1, max_col=worksheet.max_column
):
    for cell in row:
        cell.border = thin_border

workbook.save(output_filename)